<a href="https://colab.research.google.com/github/rayeeed/percentage-image/blob/main/Copy_of_ThesisPercentageRayeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



The code based on this notebook available here
https://github.com/CSAILVision/semantic-segmentation-pytorch/tree/master/notebooks

It can be run on Colab at this URL https://colab.research.google.com/github/CSAILVision/semantic-segmentation-pytorch/blob/master/notebooks/DemoSegmenter.ipynb

### Environment Setup

First, download the code and pretrained models if we are on colab.

In [ ]:
%%bash
# Colab-specific setup
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
pip install yacs 2>&1 >> install.log
git init 2>&1 >> install.log
git remote add origin https://github.com/CSAILVision/semantic-segmentation-pytorch.git 2>> install.log
git pull origin master 2>&1 >> install.log
DOWNLOAD_ONLY=1 ./demo_test.sh 2>> install.log

From https://github.com/CSAILVision/semantic-segmentation-pytorch
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master


## Imports and utility functions

We need pytorch, numpy, and the code for the segmentation model.  And some utilities for visualizing the data.

In [ ]:
# System libs
import os, csv, torch, numpy, scipy.io, PIL.Image, torchvision.transforms, glob, cv2, re
# Our libs
from mit_semseg.models import ModelBuilder, SegmentationModule
from mit_semseg.utils import colorEncode

colors = scipy.io.loadmat('data/color150.mat')['colors']
names = {}
with open('data/object150_info.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        names[int(row[0])] = row[5].split(";")[0]
        
print(names)
len(names)
def visualize_result(img, pred, index=None):
    # filter prediction class if requested
    if index is not None:
        pred = pred.copy()
        pred[pred != index] = -1
        print(f'{names[index+1]}:')
        
    # colorize prediction
    pred_color = colorEncode(pred, colors).astype(numpy.uint8)

    # aggregate images and save
    im_vis = numpy.concatenate((img, pred_color), axis=1)
    display(PIL.Image.fromarray(im_vis))

def frameImage(img):
    lst=[]
    pil_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
        std=[0.229, 0.224, 0.225])  # across a large photo dataset.
    ])
    pil_image = PIL.Image.open(img).convert('RGB')
    img_original = numpy.array(pil_image)
    img_data = pil_to_tensor(pil_image)
    singleton_batch = {'img_data': img_data[None].cuda()}
    output_size = img_data.shape[1:]

    # Run the segmentation at the highest resolution.
    with torch.no_grad():
        scores = segmentation_module(singleton_batch, segSize=output_size)
        
    # Get the predicted scores for each pixel
    _, pred = torch.max(scores, dim=1)
    pred = pred.cpu()[0].numpy()
    dim= pred.shape[0]*pred.shape[1]
    pred=pred.flatten()
    

    res = re.findall('grid-(\d+)-(\d+).jpg', os.path.basename(img))
    lst.append(int(res[0][0])) #insert grid number to the list
    lst.append(os.path.basename(img)) #insert image name to the list


    bin_arr = numpy.bincount(pred)
    predicted_classes = numpy.bincount(pred.flatten()).argsort()[::-1]
    for i in range(150):
        if i in predicted_classes:
          count = bin_arr[i]
          percentage= round(count*100/dim,2)
          lst.append(percentage) #inserting percentage values of each classes
          #print(str(names.get(i+1))  + ' is ' + str(percentage))
        else:
          #print(str(names.get(i+1))  + ' is ' + str(0.0))
          lst.append(0.0)
    df_length = len(df)
    df.loc[df_length] = lst #inserting the lst data of an image to the overall dataframe

{1: 'wall', 2: 'building', 3: 'sky', 4: 'floor', 5: 'tree', 6: 'ceiling', 7: 'road', 8: 'bed', 9: 'windowpane', 10: 'grass', 11: 'cabinet', 12: 'sidewalk', 13: 'person', 14: 'earth', 15: 'door', 16: 'table', 17: 'mountain', 18: 'plant', 19: 'curtain', 20: 'chair', 21: 'car', 22: 'water', 23: 'painting', 24: 'sofa', 25: 'shelf', 26: 'house', 27: 'sea', 28: 'mirror', 29: 'rug', 30: 'field', 31: 'armchair', 32: 'seat', 33: 'fence', 34: 'desk', 35: 'rock', 36: 'wardrobe', 37: 'lamp', 38: 'bathtub', 39: 'railing', 40: 'cushion', 41: 'base', 42: 'box', 43: 'column', 44: 'signboard', 45: 'chest', 46: 'counter', 47: 'sand', 48: 'sink', 49: 'skyscraper', 50: 'fireplace', 51: 'refrigerator', 52: 'grandstand', 53: 'path', 54: 'stairs', 55: 'runway', 56: 'case', 57: 'pool', 58: 'pillow', 59: 'screen', 60: 'stairway', 61: 'river', 62: 'bridge', 63: 'bookcase', 64: 'blind', 65: 'coffee', 66: 'toilet', 67: 'flower', 68: 'book', 69: 'hill', 70: 'bench', 71: 'countertop', 72: 'stove', 73: 'palm', 74: '

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
net_encoder = ModelBuilder.build_encoder(
    arch='resnet50dilated',
    fc_dim=2048,
    weights='ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth')
net_decoder = ModelBuilder.build_decoder(
    arch='ppm_deepsup',
    fc_dim=2048,
    num_class=150,
    weights='ckpt/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth',
    use_softmax=True)

crit = torch.nn.NLLLoss(ignore_index=-1)
segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)
segmentation_module.eval()
segmentation_module.cuda()

Loading weights for net_encoder
Loading weights for net_decoder


SegmentationModule(
  (encoder): ResnetDilated(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): SynchronizedBatchNorm2d(128, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): SynchronizedBatchNorm2d(64, eps=1

In [ ]:
import pandas as pd
df=pd.DataFrame()
df = df.join(pd.DataFrame(columns= ['grid','imagename']))
df = df.join(pd.DataFrame(columns=names.values()))
df

,grid,imagename,wall,building,sky,floor,tree,ceiling,road,bed,windowpane,grass,cabinet,sidewalk,person,earth,door,table,mountain,plant,curtain,chair,car,water,painting,sofa,shelf,house,sea,mirror,rug,field,armchair,seat,fence,desk,rock,wardrobe,lamp,bathtub,...,stool,barrel,basket,waterfall,tent,bag,minibike,cradle,oven,ball,food,step,tank,trade,microwave,pot,animal,bicycle,lake,dishwasher,screen,blanket,sculpture,hood,sconce,vase,traffic,tray,ashcan,fan,pier,crt,plate,monitor,bulletin,shower,radiator,glass,clock,flag


In [ ]:
for file in glob.glob("/content/drive/MyDrive/location 9639-13500/*.jpg"):
    print('\n'+" start of image " + str(file) +'\n')
    frameImage(file)


In [ ]:
df

,grid,imagename,wall,building,sky,floor,tree,ceiling,road,bed,windowpane,grass,cabinet,sidewalk,person,earth,door,table,mountain,plant,curtain,chair,car,water,painting,sofa,shelf,house,sea,mirror,rug,field,armchair,seat,fence,desk,rock,wardrobe,lamp,bathtub,...,stool,barrel,basket,waterfall,tent,bag,minibike,cradle,oven,ball,food,step,tank,trade,microwave,pot,animal,bicycle,lake,dishwasher,screen,blanket,sculpture,hood,sconce,vase,traffic,tray,ashcan,fan,pier,crt,plate,monitor,bulletin,shower,radiator,glass,clock,flag
0,12816,grid-12816-0.jpg,1.34,0.00,84.95,13.54,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.02,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12816,grid-12816-90.jpg,1.34,0.00,84.95,13.54,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.02,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12817,grid-12817-0.jpg,1.34,0.00,84.95,13.54,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.02,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12816,grid-12816-270.jpg,1.34,0.00,84.95,13.54,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.02,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12816,grid-12816-180.jpg,1.34,0.00,84.95,13.54,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.02,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10111,9676,grid-9676-270.jpg,26.16,0.02,0.00,6.99,0.12,44.48,0.00,0.72,8.07,0.00,0.08,0.0,0.00,0.17,0.48,0.6,0.15,0.06,0.0,3.41,0.01,1.34,0.66,0.01,0.0,0.0,0.0,0.0,0.0,0.00,1.55,0.99,0.00,0.0,0.0,0.0,0.69,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10112,9692,grid-9692-0.jpg,0.00,0.00,50.90,0.00,1.38,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,13.12,0.00,0.0,0.00,13.71,0.0,0.00,0.00,9.88,0.00,0.00,0.0,0.0,0.0,0.0,0.0,10.25,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10113,9692,grid-9692-180.jpg,0.00,1.34,51.45,0.00,5.05,0.00,0.00,0.00,0.00,38.65,0.00,0.0,0.85,0.18,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.39,0.00,0.00,2.10,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10114,9692,grid-9692-90.jpg,0.00,0.00,50.99,0.00,1.54,0.00,0.00,0.00,0.00,19.86,0.00,0.0,0.00,23.76,0.00,0.0,0.15,3.34,0.0,0.00,0.00,0.35,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0

In [ ]:
indexNames = df[ (df['sky'] == 84.95) & (df['floor'] == 13.54)].index #My dataset had blank images which would give out these values for these two parameters
df1=df.drop(indexNames)

In [ ]:
df1

,grid,imagename,wall,building,sky,floor,tree,ceiling,road,bed,windowpane,grass,cabinet,sidewalk,person,earth,door,table,mountain,plant,curtain,chair,car,water,painting,sofa,shelf,house,sea,mirror,rug,field,armchair,seat,fence,desk,rock,wardrobe,lamp,bathtub,...,stool,barrel,basket,waterfall,tent,bag,minibike,cradle,oven,ball,food,step,tank,trade,microwave,pot,animal,bicycle,lake,dishwasher,screen,blanket,sculpture,hood,sconce,vase,traffic,tray,ashcan,fan,pier,crt,plate,monitor,bulletin,shower,radiator,glass,clock,flag
56,12830,grid-12830-0.jpg,8.70,17.93,47.59,0.00,1.60,0.00,3.30,0.00,0.00,0.00,0.00,8.31,0.12,9.29,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.15,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
57,12830,grid-12830-90.jpg,17.31,23.07,42.01,0.12,0.57,0.00,0.00,0.00,0.00,0.01,0.00,0.00,7.51,4.11,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.02,0.19,0.00,0.0,0.0,0.15,0.0,0.0,0.00,0.00,0.00,1.75,0.0,0.17,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.28,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
58,12830,grid-12830-180.jpg,0.03,94.73,2.63,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.11,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.21,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01
59,12830,grid-12830-270.jpg,0.33,34.10,35.72,0.00,0.02,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.00,26.05,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.38,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
87,12838,grid-12838-0.jpg,0.37,16.71,48.18,0.00,0.45,0.00,21.34,0.00,0.00,0.00,0.00,0.33,0.78,0.41,0.00,0.0,0.00,0.00,0.0,0.00,4.96,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.02,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10111,9676,grid-9676-270.jpg,26.16,0.02,0.00,6.99,0.12,44.48,0.00,0.72,8.07,0.00,0.08,0.00,0.00,0.17,0.48,0.6,0.15,0.06,0.0,3.41,0.01,1.34,0.66,0.01,0.0,0.0,0.00,0.0,0.0,0.00,1.55,0.99,0.00,0.0,0.00,0.0,0.69,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
10112,9692,grid-9692-0.jpg,0.00,0.00,50.90,0.00,1.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.12,0.00,0.0,0.00,13.71,0.0,0.00,0.00,9.88,0.00,0.00,0.0,0.0,0.00,0.0,0.0,10.25,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
10113,9692,grid-9692-180.jpg,0.00,1.34,51.45,0.00,5.05,0.00,0.00,0.00,0.00,38.65,0.00,0.00,0.85,0.18,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.39,0.00,0.00,2.10,0.0,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
10114,9692,grid-9692-90.jpg,0.00,0.00,50.99,0.00,1.54,0.00,0.00,0.00,0.00,19.86,0.00,0.00,0.00,23.76,0.00,0.

For my project, I needed to pool 4 images of same grids together as averaged value. Ignore this segment if it's not necessary for you.

In [ ]:
blank_rows= df.shape[0]-df1.shape[0]
blank_grid= blank_rows/4
total_grid= df.shape[0]/4
percent_blank= round((blank_grid/total_grid)*100,2)
print("percentage of blank grid is: %f" %(percent_blank))

percentage of blank grid is: 78.490000


In [ ]:
grouped_df1 = df1.groupby('grid').mean() 
grouped_df1

,wall,building,sky,floor,tree,ceiling,road,bed,windowpane,grass,cabinet,sidewalk,person,earth,door,table,mountain,plant,curtain,chair,car,water,painting,sofa,shelf,house,sea,mirror,rug,field,armchair,seat,fence,desk,rock,wardrobe,lamp,bathtub,railing,cushion,...,stool,barrel,basket,waterfall,tent,bag,minibike,cradle,oven,ball,food,step,tank,trade,microwave,pot,animal,bicycle,lake,dishwasher,screen,blanket,sculpture,hood,sconce,vase,traffic,tray,ashcan,fan,pier,crt,plate,monitor,bulletin,shower,radiator,glass,clock,flag
grid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9639,3.1625,6.3175,37.8475,0.000,23.2500,0.0,2.5275,0.0,0.000,5.1925,0.0025,1.3300,0.5550,6.3050,0.0000,0.0175,0.36,1.4150,0.0,0.000,2.9125,0.1150,0.0,0.0,0.0,2.4900,0.0,0.0,0.0,0.000,0.0,0.0,0.3850,0.0,0.2425,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
9640,1.3500,0.0000,84.7600,13.840,0.0300,0.0,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0200,0.0000,0.0000,0.0000,0.00,0.0000,0.0,0.000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
9641,1.3500,0.0000,84.7600,13.840,0.0300,0.0,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0200,0.0000,0.0000,0.0000,0.00,0.0000,0.0,0.000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
9642,1.3500,0.0000,84.7600,13.840,0.0300,0.0,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0200,0.0000,0.0000,0.0000,0.00,0.0000,0.0,0.000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
9643,1.3500,0.0000,84.7600,13.840,0.0300,0.0,0.0000,0.0,0.000,0.0000,0.0000,0.0000,0.0200,0.0000,0.0000,0.0000,0.00,0.0000,0.0,0.000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12920,0.0425,4.5925,41.7950,0.000,13.3150,0.0,0.0800,0.0,0.005,8.9150,0.0000,0.0000,0.0000,18.2450,0.0075,0.0000,0.00,0.1375,0.0,0.000,0.0000,0.0300,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,11.565,0.0,0.0,1.0025,0.0,0.0550,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
12937,2.8300,39.0275,16.6300,0.130,7.4150,0.0,0.0000,0.0,0.000,27.3325,0.0000,0.1075,0.2275,0.4550,0.0000,0.0000,0.00,4.2800,0.0,0.000,0.0000,0.0025,0.0,0.0,0.0,0.0150,0.0,0.0,0.0,0.000,0.0,0.0,0.5675,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0175,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040
12976,0.0700,5.1800,44.2625,0.195,20.5375,0.0,12.7975,0.0,0.000,0.0000,0.0000,1.7600,0.6975,7.1475,0.0025,0.2575,0.00,0.0000,0.0,0

In [ ]:
grouped_df1.to_csv('/content/data/extractedData9.5-13.5.csv',index=True)